In [169]:
import ephem
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

f = open("gauquelin.txt", "rt")
data = [s.split('\t') for s in f]
header = data[0]
data = data[2:]
len(data)

8776

In [284]:
def convert_angle(s):
    sign = '+' if (s[-3]=='N' or s[-3]=='E') else '-'
    return sign + s[:-3] + ':' + s[-2:].lstrip(' ')
        
def get_planet_position(obs, planet):
    pl = planet(obs)
    house = int(np.floor(pl.az / (2*np.pi) * 12))
    sign = int(np.floor(ephem.Ecliptic(pl).lon / (2*np.pi) * 12))
    return [sign, house]

def decode_index(ind):
    p = ind / 24
    i = ind % 24
    circle = 'sign' if i < 12 else 'house'
    n = i % 12
    planets = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn']
    return planets[p] + ' in ' + circle + ' ' + str(n+1)

class Person:
    def __init__(self, entry):
        self.obs = ephem.Observer()
        hour = str(int(entry[6]) + int(entry[9]))
        self.obs.date = entry[0] + '/' + entry[1] + '/' + entry[2] + ' ' + hour + ':' + entry[7]
        self.obs.lat = convert_angle(entry[10])
        self.obs.lon = convert_angle(entry[11])
        self.occupation = entry[3]

In [228]:
n = len(data)
planets = [ephem.Sun, ephem.Moon, ephem.Mercury, ephem.Venus, ephem.Mars, ephem.Jupiter, ephem.Saturn]
features = []
for i in range(n):
    p = Person(data[i])
    features.append([])
    for planet in planets:
        features[-1].extend(get_planet_position(p.obs, planet))
        
m = 12 * len(features[0])
X = np.zeros((n, m))
y = - np.ones(n)

for i in range(n):
    for j in range(len(features[0])):
        X[i, j*12 + features[i][j]] = 1
    if data[i][3] == 'C':
        y[i] = 0
    elif data[i][3] == 'S':
        y[i] = 1
    elif data[i][3] == 'M':
        y[i] = 2

In [229]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [294]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l1', C=0.1)
scores = cross_val_score(clf, X, y, cv=10)

print scores
print 'prediction accuracy = ', np.mean(scores), ' +- ' , np.std(scores)/np.sqrt(len(scores))

[ 0.44141069  0.43913538  0.44596132  0.44646925  0.42710706  0.43507973
  0.44355758  0.41894977  0.42237443  0.45319635]
prediction accuracy =  0.437324156183  +-  0.00337569919749


In [295]:
print 'baseline accuracy = ', 1.0 * np.sum(y==1) / len(y)

baseline accuracy =  0.415109389243
